In [2]:
using LinearAlgebra
using SparseArrays
using MathProgBase
using Clp
using PyPlot

In [3]:
"""
    ratiotest(x, A, b, d)
Perform the ratio test: given the direction `d` choose
`α = argmin b -  A(x+alpha d) ≧ 0`.
"""

function ratiotest(x, A, b, d)
    min_ratio = 0.
    Atimesd = A*d
    count = 0
    for (index,aTd) in  enumerate(Atimesd)
        if aTd > 1e-8
           count += 1
           ratio  = (b[index] - dot(A[index,:],x))/aTd
           if ratio < min_ratio || count == 1
               min_ratio = ratio
           end
       end
    end
    return min_ratio
end


"""
    barydirection(x, A, b, d)
Finds Barycentric direction
"""
function barydirection(x, A, b, Anormed,cnormed)
    # Find the Opposite Barycenteric direction
    # Find indexes J such that b-Ax < ϵ
    J = findall(abs.(b-A*x) .< eps())
    cardJp1 = length(J) + 1.
    d = cnormed
    for index in J
        d += Anormed[index,:]
    end
    d /= -cardJp1
    return d
end

"""
    finddirection(x, A, b, Anormed,cnormed)
Finds Barycenteric direction
"""
function finddirection(x, A, b, Anormed,cnormed, num_var, sortedJ)
    # Find the Opposite to Circumcenter direction
    # Finds indexes J such that b-Ax < ϵ
    J = findall(abs.(b-A*x) .< 1e-8)
    lenJ = length(J)
    # If x is interior, takes direction -c
    if isempty(J)
        return -cnormed
    end
    index = 1
    while lenJ >  (num_var-1)
        filter!(x->x != sortedJ[index],J)
        lenJ = length(J)
        index +=1
    end
    X = Matrix([cnormed (Anormed[J,:])'])
    xcirc = FindCircumcentermSet(X)
    return -xcirc
end

"""
    refinesolution(x, A, b, c, num_var)
Refine solution when near the LP solution
"""


"""
mpstomatrix(mpsfile::String)
The GLPK solver is used to convert the MPS file to a problem with the matrix form
       min  dot(c, x)
subject to bl ≤ A x ≤ bu
           0 ≤ x ≤ xu
"""

function mpstomatrix(mpsfile::String)
    lp_model = MathProgBase.LinearQuadraticModel(ClpSolver())
    # Load the model from .MPS file
    MathProgBase.loadproblem!(lp_model, mpsfile)
    # Transform in vectors
    c = MathProgBase.getobj(lp_model)
    A = MathProgBase.getconstrmatrix(lp_model)
    bl = MathProgBase.getconstrLB(lp_model)
    bu = MathProgBase.getconstrUB(lp_model)
    xl = MathProgBase.getvarLB(lp_model)
    xu = MathProgBase.getvarUB(lp_model)
    return c, A, bl, bu, xl, xu
end

"""
LPtoSTDFormat(c,A,l,u,xlb,xub)
Transforms LP of format
       min  dot(c, x)
subject to l ≦  A x ≦ u
          xl ≤ x ≤ xu
into the standart format
        min  dot(c, x)
subject to  A x = b
          0 ≤ x ≤ xu
"""
function LPtoSTDFormat(c,A,l,u,xlb,xub)
    nrow, ncol = size(A)
    b = Array{Float64,1}()
    # Transform l <= Ax <= u into Ax = b
    delrows = Array{Int64,1}()
    for row = 1:nrow
            if l[row] > u[row]
                throw(error("Problem is infeasible."))
            elseif l[row] == -Inf  && u[row] == Inf #Constraint is always feasible
                push!(delrows,row)
                push!(b,Inf)      #Creates dummy b[row] just to delete at the end.
            elseif l[row] == u[row] # Constraint is l = a'x = u
                    push!(b, l[row])
            elseif l[row] > -Inf && u[row] == Inf #Constraint is  a'x >= l
                ncol += 1
                A = [A spzeros(nrow)]
                A[row,end] = -1.0 # a'x - xs = l
                push!(b, l[row]) # b <- l
                push!(c, 0.) # no cost
                push!(xlb, 0.) #xs >= 0
                push!(xub, Inf) #xs <= Inf
            elseif l[row] == -Inf && u[row] < Inf # Constraint is  a'x <= u
                ncol += 1
                A = [A spzeros(nrow)]
                A[row,end] = 1.0 # a'x + xs = u
                push!(b, u[row]) # b <- u
                push!(c, 0.) # no cost
                push!(xlb, 0.) #xs >= 0
                push!(xub, Inf) #xs <= Inf
            elseif l[row] > -Inf && u[row] < Inf # Constraint is  l <a'x < u.
                A = [A spzeros(nrow)]
                A[row,end] = -1.0 # a'x = xs
                push!(b, 0.) # b <-
                push!(c, 0.)
                push!(xlb, l[row]) # adds variable
                push!(xub, u[row])
                ncol += 1
            end
    end
    A = A[setdiff(1:end,delrows),:]
    b = deleteat!(b,delrows)
    # Transform xlb <= x <= xub  into 0 <= x <= xub
    delcols = Array{Int64,1}()
    for col = 1:ncol
        if xlb[col] > xub[col]
            throw(error("Problem is infeasible."))
        elseif xlb[col] == -Inf  && xub[col] == Inf #Free variable
            A = [A -A[:,col]] #x_i = xp - xm
            push!(c, -c[col]) # adds cost for xm
            xlb[col] = 0.
            push!(xlb, 0.) #xm >= 0
            push!(xub, Inf) #xs <= Inf
        elseif xlb[col] == xub[col] # Constraint is l = x = u
            b -= xlb[col]*A[:,col]
            push!(delcols,col)
        elseif xlb[col] > -Inf && xub[col] <= Inf
            b -= xlb[col]*A[:,col]
            xub[col] -= xlb[col]
            xlb[col] = 0.
        elseif xlb[col] == -Inf && xub[col] < Inf
            c[col] = -c[col]
            A[:,col] = -A[:,col]
            b = b - xub[col]*A[:,col]
            xlb[col] = 0.
            xub[col] = Inf
        end
    end
    A = A[:,setdiff(1:end,delcols)]
    c = c[setdiff(1:end,delcols)]
    xlb = xlb[setdiff(1:end,delcols)]
    xub = xub[setdiff(1:end,delcols)]
    nrow,ncol = size(A)
    return nrow,ncol,c,A,b,xlb,xub
end


"""
FindCircumcentermSet(X)
Finds the Circumcenter of vectors ``x_0,x_1,…,x_m``, columns of matrix ``X``,
as described in [^Behling2018a] and [^Behling2018b].
[^Behling2018]: Behling, R., Bello Cruz, J.Y., Santos, L.-R.: 
Circumcentering the Douglas–Rachford method. Numer. Algorithms. 78(3), 759–776 (2018). 
[doi:10.1007/s11075-017-0399-5](https://doi.org/10.1007/s11075-017-0399-5)
[^Behling2018]: Behling, R., Bello Cruz, J.Y., Santos, L.-R.: 
On the linear convergence of the circumcentered-reflection method. Oper. Res. Lett. 46(2), 159-162 (2018). 
[doi:10.1016/j.orl.2017.11.018](https://doi.org/10.1016/j.orl.2017.11.018)
"""
    function FindCircumcentermSet(X)
    # Finds the Circumcenter of points X = [X1, X2, X3, ... Xn]
        # println(typeof(X))
        lengthX = length(X)
        if lengthX  == 1
            return X[1]
        elseif lengthX == 2
            return .5*(X[1] + X[2])
        end
        V = []
        b = Float64[]
        # Forms V = [X[2] - X[1] ... X[n]-X[1]]
        # and b = [dot(V[1],V[1]) ... dot(V[n-1],V[n-1])]
        for ind in 2:lengthX
            difXnX1 = X[ind]-X[1]
            push!(V,difXnX1)
            push!(b,dot(difXnX1,difXnX1))
        end

       # Forms Gram Matrix
        dimG = lengthX-1
        G = diagm(b)

        for irow in 1:(dimG-1)
            for icol in  (irow+1):dimG
                G[irow,icol] = dot(V[irow],V[icol])
                G[icol,irow] = G[irow,icol]
            end
        end
        # Can we make this solution faster, or better?
        y = cholesky(G)\b
        CC = X[1]
        for ind in 1:dimG
            CC += .5*y[ind]*V[ind]
        end
        return CC
    end

FindCircumcentermSet

In [8]:
function circus(c, A, l, u, xl, xu; atol = 1e-5, max_iter = 10000)
    b= [u; -l; xu; -xl]
    AA = [A; -A; I; -I]
    return circus(c, AA, b, atol = atol, max_iter = max_iter)
end

"""
    circus(c, A, b, xu; atol = 1e-5, max_iter = 10000)
Solve the linear program problem on the form
           min  dot(c, x)
           subject to   A x = b
                      0 ≤ x ≤ xuthrow(ErrorException("test"))
where `A` is m × n. `atol` is the tolerance and `max_iter` is the
maximum number of iterations using the Circus Method.
"""

function circus(c, A, b, xu; atol = 1e-5, max_iter = 10000)
    # Including slacks
    index_slacks =  findall(xu .!=  Inf)
    num_slacks = length(index_slacks)
    num_const, num_var = size(A)
    AA = [A spzeros(num_const,num_slacks);
         sparse(collect(1:num_slacks), index_slacks, ones(Float64,num_slacks),num_slacks,num_var) I]
    bb = [b; xu[index_slacks]]
    cc = [c; zeros(num_slacks)]
    # Computes Solution for the dual
    # min dot(-[b; xu],[y; w])
    # subject to   AA^T[y; w  ≦ [c;0]
    return circus(-bb, Matrix(AA'), cc, atol = atol, max_iter = max_iter)
end

"""
    circus(c, A, b; atol = 1e-5, max_iter = 10000)
Solve the linear program problem on the form
           min  dot(c, x)
    subject to  A x ≦ b
where `A` is m × n. `atol` is the tolerance and `max_iter` is the
maximum number of iterations  using the Circus Method.
"""

function circus(c, A, b; atol = 1e-8, max_iter = 10000)
    # @assert all(b .> zero(T))

    # Set up data structures.
    num_const, num_var = size(A)
    x = zeros(num_var)  # Suppose zero is feasible
    # x[1] = 1
    cnormed = c/norm(c)
    Anormed = copy(A)
    bnormed = copy(b)
    for  i in 1:num_const
        normA = norm(Anormed[i,:])
        Anormed[i,:] /= normA
        bnormed[i] /= normA
    end
    # A = Anormed
    # b = bnormed
    # c = cnormed
    if ~all(A*x.<= b)
        error("x0 is not a feasible starting point")
    end
    # Make Phase I to find a feasible initial solution

    # Using -c as direction to go down
    # d = -c
    # min_ratio = ratiotest(x, A, b, d)
    # Taking the step
    # x += min_ratio*d
    # Begin circus iterations.
    status = :MaxIter

    iter = 0
    f = dot(c,x)
    while iter <= max_iter
        # Find Circumcenter direction
        d2 = finddirection(x, A, b, Anormed,cnormed)
        # Projects d2 into span{c}
        d1 = d2 - dot(cnormed,d2)*cnormed
        alpha1 = ratiotest(x,A,b,d1)
        alpha2 = ratiotest(x,A,b,d2)
        x1 = x + alpha1*d1
        x2 = x + alpha2*d2
        d3 = finddirection(x2, A, b, Anormed,cnormed)
        # Projects d3 into span{c}
        d3 -= dot(cnormed,d3)*cnormed
        alpha3 = ratiotest(x2,A,b,d3)
        x3 = x2 + alpha3*d3
        # Compute direction
        xm1 = .5*(x + x1)
        xm2 = .5*(x3 + x2)
        d = xm2 - xm1
        alpha = ratiotest(xm1,A,b,d)
        xnew = xm1 + alpha*d
        gaperror = norm(xnew - x)
        fnew = dot(xnew,c)
        # gaperror = norm(fnew - f)
        x = xnew
        iter += 1
        if  gaperror < atol
            # @show x
            status = :Optimal
            # iter
            x = refinesolution(x, A, b, c, num_var,atol)
            f = dot(c,x)
            break
        end
        f = dot(x,c)
    end
    return x,f,iter,status
end

"""
    circumsimplex(c, A, b; atol = 1e-5, max_iter = 10000)
Solve the linear program problem on the form
           min  dot(c, x)
    subject to  A x ≦ b
where `A` is m × n. `atol` is the tolerance and `max_iter` is the
maximum number of iterations using the accelerated Simplex by Circus.
"""

function circussimplex(c, A, b; xzero = Float64[], atol = 1e-8, max_iter = 10000)
    # @assert all(b .> zero(T))

    # Set up data structures.
    num_const, num_var = size(A)
    isempty(xzero) ?  x = zeros(num_var) : x = xzero
    # Test to verify that xzero is a feasible starting point
    if ~all(A*x - b .<= atol)
        # Make Phase to find a feasible initial solution if necessary
        # by constructing artificial problem
        println("Aqui")
        println(A*x - b )
        phaseI = true
        c = [zeros(num_var); 1.]
        maxb = maximum(abs.(b))
        x = [zeros(num_var); maxb]
        b = [b; 0.]
        art_col = spzeros(num_const)
        signb = findall(sign.(b).< 0)
        art_col[signb] = -ones(length(signb))
        A = [A art_col;
            spzeros(num_var)'  -1.]
            num_var +=1
            num_const +=1
    else
        phaseI = false
    end
    cnormed = c/norm(c)
    Anormed = copy(A)
    bnormed = copy(b)
    for  i in 1:num_const
        normA = norm(Anormed[i,:])
        Anormed[i,:] /= normA
        bnormed[i] /= normA
    end
    sortedJ = sortperm((Anormed*cnormed),rev=true)
    # Refine feasible point to find a vertex
    x = refinesolution(x, A, b, c, num_var, atol)
    iter = 0
    f = dot(c,x)
    status = :Optimal

    while iter <= max_iter
        iter += 1
        # Find Circumcenter direction
        d = finddirection(x, A, b, Anormed,cnormed, num_var,sortedJ)
        println(d)
        alpha = ratiotest(x,A,b,d)
        xnew = x + alpha*d
        xnew = refinesolution(xnew, A, b, c, num_var,atol)
        println("Indices das Ativas")
        println(findall(b-A*xnew .<=atol))
        println("Ativas")
        println(Array(xnew))
        gaperror = norm(xnew - x)
        fnew = dot(xnew,c)
        # gaperror = norm(fnew - f)
        x = xnew
        f = dot(c,x)
        if  gaperror < atol
            # @show x
            # status = :Optimal
            # iter
            # x = refinesolution(x, A, b, c, num_var,atol)
            # f = dot(c,x)
            # return x,f,iter,status
            break
        end
    end
    if iter >= max_iter
        status = :MaxIter
    end
    return x,f,iter,status
end
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
xo = [1,-1,0.0]

circussimplex(c, A, b,xzero=xo)

index_active = findall(b - A * x .< 1.0e-8) = [2]
d = -c + (A[index_active, :])' * lambda = [0.3333333333333333, 0.3333333333333333, -0.6666666666666667]
alpha = ratiotest(x, A, b, d) = 2.9999999999999996
[1.0, -1.0, 0.0]
xnew = x + alpha * d = [1.9999999999999998, -2.220446049250313e-16, -2.0]
d = -c + (A[index_active, :])' * lambda = [0.5, 0.0, -0.5]
alpha = ratiotest(x, A, b, d) = 4.0
[1.9999999999999998, -2.220446049250313e-16, -2.0]
xnew = x + alpha * d = [4.0, -2.220446049250313e-16, -4.0]


PosDefException: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [5]:
c = [-4.; -3]
A = Matrix([2. 1 2; 3 3 1]')
b = [4.; 3.; 3.]
sol = [1.25, .5]
# circussimplex(c, A, b)

2-element Array{Float64,1}:
 1.25
 0.5

In [7]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     1. 0. 0.]
b = [0,0,4.,0.]
xo = [1,-1,0.0]

function refinesolution(x, A, b, c, num_var, atol)
    @show index_active = findall(b - A*x .<  1e-8)
    num_active = length(index_active)
    iter = 0
    while num_active < num_var
       iter += 1
       if iszero(num_active)
           alpha = ratiotest(x,A,b,-c)
           x = x - alpha*c
           index_active = findall(b - A*x.<= 1e-8)
           num_active = length(index_active)
       end
       aFact = lu(A[index_active,:]*A[index_active,:]')
       lambda = aFact.L\(A[index_active,:]*c)
       lambda = aFact.U\lambda
       @show d = -c +  A[index_active,:]'*lambda
       # if norm(d) ≈ 0
       #     break
       # end
       @show alpha = ratiotest(x,A,b,d)
        println(x)
       @show xnew = x + alpha*d
       if norm(xnew - x) < atol
           return xnew
       end
       x = xnew
       index_active = findall(b - A*x .<  1e-8)
       num_active = length(index_active)
    end
    return x
end
x_ref = refinesolution(xo, A, b, c, 3, 1e-8)

index_active = findall(b - A * x .< 1.0e-8) = [2, 4]
d = -c + (A[index_active, :])' * lambda = [1.1102230246251565e-16, 0.5, -0.5]
alpha = ratiotest(x, A, b, d) = 2.0000000000000004
[1.0, -1.0, 0.0]
xnew = x + alpha * d = [1.0000000000000002, 2.220446049250313e-16, -1.0000000000000002]


3-element Array{Float64,1}:
  1.0000000000000002
  2.220446049250313e-16
 -1.0000000000000002

In [9]:
xo = [2,-1,-1.0]
pygui(true)
px= [0,4 ,4 ,0 ,0,0,0 ,0,4,4,4 ,4]
py= [0,0 ,-4,-4,0,4,-4,4,4,-4,0,4]
pz= [0,-4,0 ,4 ,0,4,4 ,4,0,0,-4,0]
plt= PyPlot.plot3D(px,py,pz)
PyPlot.scatter3D([x_ref[1]],[x_ref[2]],[x_ref[3]])
PyPlot.plot3D([xo[1],xo[1]+1],[xo[2],xo[2]+1],[xo[3],xo[3]-2])

1-element Array{PyCall.PyObject,1}:
 PyObject <mpl_toolkits.mplot3d.art3d.Line3D object at 0x7f0f92128ac0>

### Problema em formato Simplex

In [4]:
function SimplexFromBFS(c,A,b,initial_bfs;max_iterations=100,index_bfs=[0],index_nfs = [0])
    # Initial setup
    e  = 10^-5
    B  = findall(initial_bfs .> 0+e)
    N  = findall(initial_bfs .<= 0+e)
    if size(A[:,B])[1] != size(A[:,B])[2]
        B = index_bfs
        N = index_nfs
    end
    @show B
    xn = initial_bfs[N]; xb = initial_bfs[B];
    
    # Simplex pivoting iteration
    for i = 1:max_iterations
        Ab = A[:,B]; An = A[:,N]; cb = c[B]; cn = c[N]
        p  = inv(Ab)*b
        Q  = -inv(Ab)*An
        r  = (cb'*Q + cn')'
        if all(r.<= 0)
            x_final = vcat(hcat(B,p),hcat(N,zeros(length(N))))
            x_final = x_final[sortperm(x_final[:,1]),:]
            return x_final
        end
        zo = cb'*p
#         z  = zo + r'*xn
        index_in =findmax(r)[2]
        x_in = N[index_in]
        if any(Q[:,index_in] .< 0)
            coef_entering = -p./Q[:,index_in] 
            q_neg_index   = findall(Q[:,index_in] .< 0)
            index_out     =findfirst(coef_entering .== findmin(coef_entering[q_neg_index])[1])
            x_out     = B[index_out]
            B[index_out] = x_in
            N[index_in]  = x_out
        else
            
            error("Unbounded")
        end
        println(x_in,"   ",x_out)
    end
    x_final = vcat(hcat(B,p),hcat(N,zeros(length(N))))
    x_final = x_final[sortperm(x_final[:,1]),:]
    return x_final
end

c = [0.,0,0,1,-1,0,0,0]
A = [-1 1  -1 -1 1 1 0 0.;
     -1 -1 1  -1 1 0 1 0;
     1. 0  0  0  0 0 0 1]
b  = [0,0,4.]
xo = [0.,0,0,0,0,0,0,4]


SimplexFromBFS(-c,A,b,xo,index_bfs=[6,7,8],index_nfs=[1,2,3,4,5])

B = [6, 7, 8]
5   6
1   8
3   7


8×2 Array{Float64,2}:
 1.0  4.0
 2.0  0.0
 3.0  0.0
 4.0  0.0
 5.0  4.0
 6.0  0.0
 7.0  0.0
 8.0  0.0

#### Algoritmo de Cross Over

In [326]:
# Teste inicial
c = [0.,0,0,1,-1,0,0,0]
A = [-1 1  -1 -1 1 1 0 0.;
     -1 -1 1  -1 1 0 1 0;
     1. 0  0  0  0 0 0 1]
b  = [0,0,4.]
x = [1,2,1,0,0,0,2,3]

Bi = [1,2,3,7,8]
Mi = [2,7,8]
Bi_Mi = setdiff(Bi,Mi)
M  = A[:,Mi]
j = 1
z = inv(M)*A[:,j]
l = filter(e->(!(e in Mi) && e!=j),Bi)
xm = x[Mi]

# M*xm + A[:,j]*x[j] + A[:,l]*x[l] # =  Ax 
alpha = 1
# se alpha = 1, entao xm <- xm + z*xj    e fazemos xj = 0, assim temos um novo x
xm = xm + z*x[j]
x[j] = 0
x[Mi]= xm

j = 3
z = inv(M)*A[:,j]
xm + z*x[j]
alpha = 1
xm = xm + z*x[j]
x[j] = 0
x[Mi]= xm

println(A*x,c'*x)

[0.0, 0.0, 4.0]0.0


In [334]:
# Iniciando em outro ponto
function find_alpha_max(w,xm)
    alpha = 0
    for i in 1:size(xm)[1]
        if w[i] < 0
            alpha_parcial = xm[i]/w[i]
        else
            alpha_parcial = -xm[i]/w[i]
        end
        if alpha_parcial > alpha
            alpha = alpha_parcial
        end
    end

    if alpha > 1 || alpha == 0
        alpha = 1
    end
    return alpha
end

findmaxk(x,k) = sortperm(x)[end-k+1:end]

c = [0.,0,0,1,-1,0,0,0]
A = [-1 1  -1 -1 1 1 0 0.;
     -1 -1 1  -1 1 0 1 0;
     1. 0  0  0  0 0 0 1]
b  = [0,0,4.]

x = [2,1,2,1,2,2,0,2]
Bi = findall(e -> e>0, x)
m  = size(A)[1]
Mi = sort(findmaxk(x,m))
Bi_Mi = setdiff(Bi,Mi)
M  = A[:,Mi]

for j in Bi_Mi[1:4]
    z = inv(M)*A[:,j]
    l = filter(e->(!(e in Mi) && e!=j),Bi)
    xm = x[Mi]
    w = z*x[j]
    alpha = find_alpha_max(w,xm)
    if alpha == 1
        xm = xm + z*x[j]
        x[j] = 0
        x[Mi]= xm
    else
        i_pivot = findfirst(xm + alpha*w .== 0)
        xm = xm + z*x[j]*alpha
        x[Mi]= xm
        x[j] = (1-alpha)*x[j]
        Mi[i_pivot] = j
        sort!(Mi)
        M = A[:,Mi]
    end
end
x

8-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 4

1